In [64]:
import time
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm 

In [65]:
data = pd.read_csv("../paruvendu_api/outputs/raw_dataset.csv")
data.head()

,location,version,price,year,mileage,fuel_type,emission,transmission,door_nb,technical_power,seat_nb,actual_power,body_type
0,86.0,dacia/duster,17600.0,2019,18300.0,essence,NaN,manuelle,4.0,7.0,5.0,130.0,4x4
1,59.0,mini/cooper,24990.0,2018,47500.0,essence,119.0,automatique,2.0,10.0,4.0,192.0,berline
2,53.0,nissan/qashqai,8490.0,2013,166400.0,diesel,129.0,manuelle,4.0,6.0,5.0,110.0,break
3,1.0,nissan/qashqai,16900.0,2016,61500.0,diesel,128.0,automatique,4.0,7.0,5.0,130.0,4x4
4,30.0,mini/cooper,17900.0,2018,58529.0,diesel,99.0,manuelle,4.0,5.0,5.0,95.0,berline


In [66]:
df = data.copy()

In [67]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15552 entries, 0 to 15551
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location         15534 non-null  float64
 1   version          15552 non-null  object 
 2   price            15542 non-null  float64
 3   year             15552 non-null  int64  
 4   mileage          15544 non-null  float64
 5   fuel_type        14492 non-null  object 
 6   emission         9052 non-null   float64
 7   transmission     11061 non-null  object 
 8   door_nb          15102 non-null  float64
 9   technical_power  14779 non-null  float64
 10  seat_nb          10239 non-null  float64
 11  actual_power     7602 non-null   float64
 12  body_type        13197 non-null  object 
dtypes: float64(8), int64(1), object(4)
memory usage: 1.5+ MB


In [68]:
df.isna().sum()

location             18
version               0
price                10
year                  0
mileage               8
fuel_type          1060
emission           6500
transmission       4491
door_nb             450
technical_power     773
seat_nb            5313
actual_power       7950
body_type          2355
dtype: int64

## TRAINTEST - NETTOYAGE - ENCODAGE

### Nettoyage

In [69]:
df_agg = df.groupby('version', as_index=False).median()

In [70]:
df.isna().sum()

location             18
version               0
price                10
year                  0
mileage               8
fuel_type          1060
emission           6500
transmission       4491
door_nb             450
technical_power     773
seat_nb            5313
actual_power       7950
body_type          2355
dtype: int64

In [71]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=False)

actual_power       0.511188
emission           0.417953
seat_nb            0.341628
transmission       0.288773
body_type          0.151427
fuel_type          0.068158
technical_power    0.049704
door_nb            0.028935
location           0.001157
price              0.000643
mileage            0.000514
version            0.000000
year               0.000000
dtype: float64

In [72]:
# RESTART TON KERNEL BLÉRO

loop_length = len(df)
df.fillna(-1.0, inplace=True)

for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    line_agg = df_agg.loc[df_agg['version'] == line['version']]
    if line['emission'] == -1.0:
        n_emission = line_agg['emission'].item()
        line['emission'] = n_emission
    if line['seat_nb'] == -1.0:
        n_seat_nb = line_agg['seat_nb'].item()
        line['seat_nb'] = n_seat_nb
    if line['door_nb'] == -1.0:
        n_door_nb = line_agg['door_nb'].item()
        line['door_nb'] = n_door_nb
    if line['actual_power'] == -1.0:
        n_actual_power = line_agg['actual_power'].item()
        line['actual_power'] = n_actual_power
    if line['technical_power'] == -1.0:
        n_technical_power = line_agg['technical_power'].item()
        line['technical_power'] = n_technical_power
    df.iloc[i] = line
df.replace(-1.0, None, inplace=True)

100%|██████████| 15552/15552 [00:39<00:00, 391.09it/s]


In [73]:
df.isna().sum()

location             18
version               0
price                10
year                  0
mileage               8
fuel_type          1060
emission             44
transmission       4491
door_nb             100
technical_power       0
seat_nb              60
actual_power        153
body_type          2355
dtype: int64

In [74]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=False)

transmission       0.288773
body_type          0.151427
fuel_type          0.068158
actual_power       0.009838
door_nb            0.006430
seat_nb            0.003858
emission           0.002829
location           0.001157
price              0.000643
mileage            0.000514
version            0.000000
year               0.000000
technical_power    0.000000
dtype: float64

In [75]:
df.fillna(-1.0, inplace=True)
transmission_list = df['transmission'].unique()
transmission_list = transmission_list[0:2]
body_type_list_raw = df['body_type'].unique()
model_list = df['version'].unique()
print(transmission_list)
print(body_type_list_raw)
print(model_list)
df.isna().sum()

['manuelle' 'automatique']
['4x4' 'berline' 'break' 'cabriolet' -1.0 'coupé' 'monospace']
['dacia/duster' 'mini/cooper' 'nissan/qashqai' 'fiat/500' 'citroen/ds3'
 'volkswagen/golf' 'citroen/ds7' 'opel/corsa' 'dacia/sandero' 'seat/leon'
 'mercedes/classe-a' 'toyota/yaris' 'peugeot/208' 'toyota/c-hr'
 'bmw/serie-1' 'seat/ibiza' 'ford/fiesta' 'audi/a3' 'peugeot/3008'
 'ford/kuga' 'nissan/juke' 'fiat/500-x' 'mercedes/classe-c'
 'mini/countryman' 'ford/focus' 'bmw/serie-3' 'toyota/aygo' 'peugeot/2008'
 'volvo/xc60' 'peugeot/308' 'opel/mokka' 'nissan/micra'
 'land-rover/range-rover-evoque' 'renault/captur' 'volkswagen/tiguan'
 'opel/astra' 'porsche/911' 'hyundai/tucson' 'kia/sportage' 'renault/clio'
 'volkswagen/polo' 'seat/arona' 'audi/a1' 'toyota/corolla' 'bmw/x1'
 'audi/a4' 'seat/ateca' 'fiat/tipo' 'audi/q3' 'opel/crossland-x'
 'mercedes/classe-cla' 'mini/cooper-d' 'mazda/mx-5' 'renault/kadjar'
 'kia/ceed' 'opel/grandland-x' 'toyota/rav-4' 'mercedes/classe-gla'
 'mercedes/classe-glc' 'nis

location           0
version            0
price              0
year               0
mileage            0
fuel_type          0
emission           0
transmission       0
door_nb            0
technical_power    0
seat_nb            0
actual_power       0
body_type          0
dtype: int64

In [76]:
dict_model_transmission = dict()
for model in model_list:
    dict_model_transmission[model] = dict()
    for transmission in transmission_list:
        dict_model_transmission[model][transmission] = 0

loop_length = len(df)
for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    model = line['version']
    transmission = line['transmission']
    if transmission != -1:
        dict_model_transmission[model][transmission] += 1

100%|██████████| 15552/15552 [00:04<00:00, 3150.32it/s]


In [77]:
for key in dict_model_transmission.keys():
    dict_model_transmission[key] = max(dict_model_transmission[key],key=dict_model_transmission[key].get)

In [78]:
body_type_list_raw = df['body_type'].unique()
print(body_type_list_raw)

['4x4' 'berline' 'break' 'cabriolet' -1.0 'coupé' 'monospace']


In [79]:
body_type_list = []
for i in range(len(body_type_list_raw)):
    value = body_type_list_raw[i]
    if value != -1:
        body_type_list.append(value)
print(body_type_list)

['4x4', 'berline', 'break', 'cabriolet', 'coupé', 'monospace']


In [80]:
dict_model_body_type = dict()
for model in model_list:
    dict_model_body_type[model] = dict()
    for body_type in body_type_list:
        dict_model_body_type[model][body_type] = 0

loop_length = len(df)
for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    model = line['version']
    body_type = line['body_type']
    if body_type != -1:
        dict_model_body_type[model][body_type] += 1

100%|██████████| 15552/15552 [00:05<00:00, 2660.37it/s]


In [81]:
for key in dict_model_body_type.keys():
    dict_model_body_type[key] = max(dict_model_body_type[key],key=dict_model_body_type[key].get)

In [82]:
fuel_type_list_raw = df['fuel_type'].unique()
print(fuel_type_list_raw)

['essence' 'diesel' -1.0]


In [83]:
fuel_type_list = fuel_type_list_raw[0:2]
print(fuel_type_list)

['essence' 'diesel']


In [84]:
dict_model_fuel_type = dict()
for model in model_list:
    dict_model_fuel_type[model] = dict()
    for fuel_type in fuel_type_list:
        dict_model_fuel_type[model][fuel_type] = 0

loop_length = len(df)
for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    model = line['version']
    fuel_type = line['fuel_type']
    if fuel_type != -1:
        dict_model_fuel_type[model][fuel_type] += 1

100%|██████████| 15552/15552 [00:04<00:00, 3169.63it/s]


In [85]:
for key in dict_model_fuel_type.keys():
    dict_model_fuel_type[key] = max(dict_model_fuel_type[key],key=dict_model_fuel_type[key].get)

In [86]:
loop_length = len(df)
for i in tqdm(range(loop_length)):
    line = df.iloc[i].copy()
    model = line['version']
    if line['transmission'] == -1:
        n_transmission = dict_model_transmission[model]
        line['transmission'] = n_transmission
    if line['body_type'] == -1:
        n_body_type = dict_model_body_type[model]
        line['body_type'] = n_body_type
    if line['fuel_type'] == -1:
        n_fuel_type = dict_model_fuel_type[model]
        line['fuel_type'] = n_fuel_type
    df.iloc[i] = line

100%|██████████| 15552/15552 [00:26<00:00, 576.37it/s]


In [87]:
df.replace(-1,None,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15552 entries, 0 to 15551
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location         15534 non-null  object 
 1   version          15552 non-null  object 
 2   price            15542 non-null  object 
 3   year             15552 non-null  int64  
 4   mileage          15544 non-null  object 
 5   fuel_type        15552 non-null  object 
 6   emission         15508 non-null  object 
 7   transmission     15552 non-null  object 
 8   door_nb          15452 non-null  object 
 9   technical_power  15552 non-null  float64
 10  seat_nb          15492 non-null  object 
 11  actual_power     15399 non-null  object 
 12  body_type        15552 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.5+ MB


In [88]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=False)

actual_power       0.009838
door_nb            0.006430
seat_nb            0.003858
emission           0.002829
location           0.001157
price              0.000643
mileage            0.000514
version            0.000000
year               0.000000
fuel_type          0.000000
transmission       0.000000
technical_power    0.000000
body_type          0.000000
dtype: float64

In [89]:
df.isna().sum()

location            18
version              0
price               10
year                 0
mileage              8
fuel_type            0
emission            44
transmission         0
door_nb            100
technical_power      0
seat_nb             60
actual_power       153
body_type            0
dtype: int64

In [90]:
df.dropna(inplace=True)

In [91]:
df.isna().sum()

location           0
version            0
price              0
year               0
mileage            0
fuel_type          0
emission           0
transmission       0
door_nb            0
technical_power    0
seat_nb            0
actual_power       0
body_type          0
dtype: int64

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15276 entries, 0 to 15551
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   location         15276 non-null  object 
 1   version          15276 non-null  object 
 2   price            15276 non-null  object 
 3   year             15276 non-null  int64  
 4   mileage          15276 non-null  object 
 5   fuel_type        15276 non-null  object 
 6   emission         15276 non-null  object 
 7   transmission     15276 non-null  object 
 8   door_nb          15276 non-null  object 
 9   technical_power  15276 non-null  float64
 10  seat_nb          15276 non-null  object 
 11  actual_power     15276 non-null  object 
 12  body_type        15276 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.6+ MB


In [93]:
df.to_csv('../paruvendu_api/outputs/clean_dataset.csv', index=False, encoding="utf-8")

In [94]:
a

NameError: name 'a' is not defined

## FEATURE SELECTION

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
selector = VarianceThreshold(0.6)
selector.fit_transform(X_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
selector = SelectKBest(f_regression, k=2)
selector.fit_transform(X_train, y_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.svm import SVR

In [ ]:
a

In [ ]:
estimator = SVR(kernel="linear")
selector = RFE(estimator, n_features_to_select=2, step=1)
selector = selector.fit(X_train, y_train)
cols = selector.get_support(indices=True)
features_df_new = X_train.iloc[:, list(cols)]
features_df_new.head()